In [1]:
from multiprocessing import Pool
import mnist_conv_model as model
import traceback
import sys

In [3]:
dict = {}
dict2 = {'a' : 1}
dict[1] = dict2
print dict

{1: {'a': 1}}


In [2]:
from scipy.stats import linregress
def fit_line1(x, y):
    """Return slope, intercept of best fit line."""
    # Remove entries where either x or y is NaN.
    clean_data = pd.concat([x, y], 1).dropna(0) # row-wise
    (_, x), (_, y) = clean_data.iteritems()
    slope, intercept, r, p, stderr = linregress(x, y)
    return slope, intercept # could also return stderr

In [19]:
import pandas as pd
df = pd.DataFrame({'x': range(2), 'y' : [1,1]})
s, i = fit_line1(df['x'], df['y'])
print s

0.0


In [25]:
a = float('NaN')
b = 0.0
print a != b

True


In [2]:
NUM_PROCESSES = 2

In [3]:
# create grid search parameter list
def create_params_list():
    params_list = []

    filter_sizes = [6]
    conv1_depths = [64]
    conv2_depths = [64, 128]
    fc_depths = [64, 128]

    for f in range(len(filter_sizes)):
        filter_size = filter_sizes[f]
        for conv1 in range(len(conv1_depths)):
            conv1_depth = conv1_depths[conv1]
            for conv2 in range(len(conv2_depths)):
                conv2_depth = conv2_depths[conv2]
                for fc in range(len(fc_depths)):
                    fc_depth = fc_depths[fc]
                    params = {"filter_size": filter_size, "conv1_depth" : conv1_depth,\
                              "conv2_depth": conv2_depth, "fc_depth": fc_depth}
                    params_list.append(params)
    return params_list           

In [4]:
def mnist_cnn(repeats):
    global iteration
    global mnist
    
    assign = repeats % NUM_PROCESSES
    #print "repeat: " + str(repeats) + ", iteration: " + str(iteration) + ", assign: " + str(assign)
    iteration += 1
    
    params = params_list.pop(0) # for FIFO
    #print "Try to learn with : " + str(params)
    eval_device_id = '/gpu:' + str(NUM_PROCESSES * assign)
    train_device_id = '/gpu:' + str(NUM_PROCESSES * assign + 1)
    return model.learn(mnist, params, train_dev=train_device_id, eval_dev=eval_device_id, progress=False, epochs=1)

In [5]:
mnist = model.download_dataset()
params_list = create_params_list()
iteration = 0
num_repeats = len(params_list)
print "total tests: " + str(num_repeats)

('Extracting', 'data/train-images-idx3-ubyte.gz')
('Extracting', 'data/train-labels-idx1-ubyte.gz')
('Extracting', 'data/t10k-images-idx3-ubyte.gz')
('Extracting', 'data/t10k-labels-idx1-ubyte.gz')
total tests: 4


In [6]:
def grid_search():
    pool = Pool(processes=2)             
    try:
        result = pool.apply_async(mnist_cnn, [num_repeats])    
        print result.get(timeout=None)           
        print pool.map(mnist_cnn, range(num_repeats))
    except:
        e = sys.exc_info()[0]
        traceback.print_exc()
        print e

In [7]:
if __name__ == '__main__':
    grid_search()

("Params: {'filter_size': 6, 'fc_depth': 64, 'conv1_depth': 64, 'conv2_depth': 64}",)
("KVParams: {'epochs': 1, 'progress': False, 'train_dev': '/gpu:1', 'eval_dev': '/gpu:0'}",)
('show progress: False',)
('Logging test errors at test.csv',)
('Training device id: /gpu:1',)
('Evaluation device id: /gpu:0',)
('Epochs: 1',)
('optimizer: Momentum',)
('epoch num: 1, total steps: 859',)
('0',)
('100',)
('200',)
('300',)
('400',)
('500',)
('600',)
('700',)
('800',)
('Test error: 2.2%',)
('Result: 2.23, Duration: 8.8593788147',)
("Params: {'filter_size': 6, 'fc_depth': 128, 'conv1_depth': 64, 'conv2_depth': 64}",)
("KVParams: {'epochs': 1, 'progress': False, 'train_dev': '/gpu:3', 'eval_dev': '/gpu:2'}",)
("Params: {'filter_size': 6, 'fc_depth': 64, 'conv1_depth': 64, 'conv2_depth': 64}",)
('show progress: False',)
("KVParams: {'epochs': 1, 'progress': False, 'train_dev': '/gpu:1', 'eval_dev': '/gpu:0'}",)
('Logging test errors at test.csv',)
('show progress: False',)
('Logging test errors at 

In [3]:
import pickle
obj = {'name' : "hello"}
with open('entry.pickle', 'wb') as f:
    pickle.dump(obj, f)  

In [4]:
with open('entry.pickle', 'rb') as f:
    entry = pickle.load(f)
    print str(entry)

{'name': 'hello'}


In [5]:
devs = 4
dev_ids = [i for i in range(devs)]
dev_ids.reverse()
print dev_ids

[3, 2, 1, 0]
